In [4]:
import numpy as np
import sys

In [9]:
print(np.__version__)
print(sys.version)

1.19.2
3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]


In [8]:
a = np.arange(16, dtype=np.int64).reshape((2,2,4,))
print(a)
print(a.strides)

[[[ 0  1  2  3]
  [ 4  5  6  7]]

 [[ 8  9 10 11]
  [12 13 14 15]]]
(64, 32, 8)


# Dimension index is a pointer
1. Entities are in the contiguous one-dimensional segment of computer memory
2. Dimension is a pointer schema to dissect the one-dimensional segment into N dimensional structure.
3. Index in the dimension d points to the block starting at ((index-1) * strides[d-1]).

Imagine the tape in a turing machine. To have n-dimensional structure in the flat sequence storage, you need a hierarchical pointer array structure, which is the np.ndarray.


## Single dimensional indices 
```
np.ndarray[
  [0,1,2]   # <--- [] is to specify indeices in the dimension 
]
```

## Multi dimensional indices
```
a[
  (0,1,2)  # <--- () is to specify indeices in each dimension. Same with a[0][1][2].
]

This is source of confusion because it is the same with:
a[
  (0),
  (1),
  (2)
  ...
]

```


# Solution to indexing confusions
Beware which you are working on → Within--dimension or Inter-dimensions

* Within-dimension → Stick to ```[ , , ,]```
* Inter-dimensions →  Stick to ```( , , , )```

### Single dimensional indices

In [41]:
a = np.arange(24, dtype=np.int64).reshape((3,2,4,))
print(a)
axix3_indices = [0,1]     # Select the blocks 0 and 1 on axis 3 
axis3_blocks = a[
    axix3_indices, # indices to the axix3 blocks
    ::,            # all blocks in axis2
    ::             # all entities in axis3 (real entities)
]
print(axis3_blocks)
print("Is this copy/fancy indexing? {}".format(axis3_blocks.base is None))

[[[ 0  1  2  3]
  [ 4  5  6  7]]

 [[ 8  9 10 11]
  [12 13 14 15]]

 [[16 17 18 19]
  [20 21 22 23]]]
[[[ 0  1  2  3]
  [ 4  5  6  7]]

 [[ 8  9 10 11]
  [12 13 14 15]]]
Is this copy/fancy indexing? True


### Multi dimensional indices

In [33]:
a = np.arange(24, dtype=np.int64).reshape((3,2,4,))
print(a)
index_at_each_dimension = (0,1)     # Select the blocks 0 in axis n, and 1 on axis n-1. Same with a[1][2]
block_from_dimensions = a[
    index_at_each_dimension
]
print(block_from_dimensions)
print(a[0][1])

[[[ 0  1  2  3]
  [ 4  5  6  7]]

 [[ 8  9 10 11]
  [12 13 14 15]]

 [[16 17 18 19]
  [20 21 22 23]]]
[4 5 6 7]
[4 5 6 7]


More confusion with (0, (0,1)). This is the same with:
```
a[
  (0),
  (0,1)
]
```
This is also the same with
```
a[0][0:2]
```

In [34]:
a = np.arange(24, dtype=np.int64).reshape((3,2,4,))
print(a)
indices_at_each_dimension = (0,(0,1))     # Select the blocks 0 in axis n. Then0 and 1 on axis n-1.
blocks_from_dimensions = a[
    indices_at_each_dimension
]
print(blocks_from_dimensions)
print(a[0][0:2])

[[[ 0  1  2  3]
  [ 4  5  6  7]]

 [[ 8  9 10 11]
  [12 13 14 15]]

 [[16 17 18 19]
  [20 21 22 23]]]
[[0 1 2 3]
 [4 5 6 7]]
[[0 1 2 3]
 [4 5 6 7]]


In [35]:
a[0, (0,1)]

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

---

# View or Copy
Check the base attribute. If it is Not None (share a base = view -> Reference the view), it is view.

In [43]:
Z = np.random.uniform(0,1,(5,5))
Z1 = Z[:3,:]
Z2 = Z[[0,1,2], :]
print(Z1.base is Z)    # view is True if it shares the base
print(Z2.base is Z)    # View? False if does not refer to the same source
print(Z2.base is None) # Copy? True if it is None, refer to no source.

True
False
True


## Temporal copy

In [49]:
import numpy as np
import timeit

setup = """
import numpy as np

X = np.ones(1000000, dtype=np.int)
Y = np.ones(1000000, dtype=np.int)
"""
iterations = 100000
elepased = timeit.timeit(
    stmt="X = X + 2*Y",
    setup=setup,
    number=iterations
)
elepased / iterations * 1e6  # micro sec

2777.6011788399774

In [50]:
elepased = timeit.timeit(
    stmt="X += 2*Y",
    setup=setup,
    number=iterations
)
elepased / iterations * 1e6  # micro sec

2703.514808419932

In [51]:
elepased = timeit.timeit(
    stmt="np.add(X, 2*Y, out=X)",
    setup=setup,
    number=iterations
)
elepased / iterations * 1e6  

2719.944044290023

In [52]:
# # + is slower than *
elepased = timeit.timeit(
    stmt="Y+=Y; np.add(X, Y, out=X)",
    setup=setup,
    number=iterations
)
elepased / iterations * 1e6  # micro sec

2643.7640607499634

In [53]:
# + is slower than *
elepased = timeit.timeit(
    stmt="np.add(Y, Y, out=Y); np.add(X, Y, out=X)",
    setup=setup,
    number=iterations
)
elepased / iterations * 1e6  # micro sec

2634.768036000023

In [54]:
elepased = timeit.timeit(
    stmt="np.add(X, Y, out=X); np.add(X, Y, out=X)",
    setup=setup,
    number=iterations
)
elepased / iterations * 1e6  

2660.2724975399906